# 01-基础对话（多轮）测试

## GLM-4.7-Flash 模型能力

根据官方文档，GLM-4.7-Flash 具备以下能力：

| 能力 | 支持情况 | 说明 |
|:---|:---:|:---|
| **思考模式** | ✅ 支持 | 提供多种思考模式，覆盖不同任务需求 |
| **流式输出** | ✅ 支持 | 支持实时流式响应，提升用户交互体验 |
| **Function Call** | ✅ 支持 | 强大的工具调用能力 |
| **上下文缓存** | ✅ 支持 | 智能缓存机制，优化长对话性能 |
| **结构化输出** | ✅ 支持 | 支持 JSON 等结构化格式输出 |
| **MCP** | ✅ 支持 | 可灵活调用外部 MCP 工具 |

## 模型规格

| 规格项 | 数值 |
|:---|:---|
| 输入模态 | 文本 |
| 输出模态 | 文本 |
| 上下文窗口 | 200K |
| 最大输出 Tokens | 128K |

## 测试内容
1. 环境配置
2. 基础对话测试
3. 多轮对话测试
4. 思考模式测试（thinking + reasoning_content）
5. 流式输出测试
6. 错误处理

## 1. 环境配置

In [4]:
# 安装依赖
# !pip install openai python-dotenv

In [5]:
import os
import time
from openai import OpenAI

# 加载环境变量
try:
    from dotenv import load_dotenv
    load_dotenv()
    api_key = os.getenv('VITE_ZHIPU_API_KEY')
except:
    api_key = None

# 如果环境变量中没有，使用默认 API Key
if not api_key:
    api_key = "f7ec61a7856a4f4389e037eb588c69df.0pZgcFlTCk9GHCXM"

BASE_URL = "https://open.bigmodel.cn/api/paas/v4"

# 初始化客户端
client = OpenAI(
    api_key=api_key,
    base_url=BASE_URL
)

print("✅ 客户端初始化成功！")
print(f"API Key: {api_key[:10]}...{api_key[-10:]}")

✅ 客户端初始化成功！
API Key: f7ec61a785...lTCk9GHCXM


## 2. 基础对话测试

In [6]:
# 基础对话 - 简单问候
response = client.chat.completions.create(
    model="glm-4.6v-flash",
    messages=[
        {"role": "user", "content": "你好，请介绍一下自己"}
    ]
)

print("🤖 模型回复：")
print(response.choices[0].message.content)
print(f"\n📊 Token 使用：{response.usage.total_tokens} (输入: {response.usage.prompt_tokens}, 输出: {response.usage.completion_tokens})")

🤖 模型回复：

你好！我是由智谱AI训练的GLM大语言模型，是一款人工智能助手。我能够理解并回应中文（以及其他多种语言）的提问，帮助你解答问题、提供信息、进行对话等。我一直在不断学习和优化，以更好地为用户提供帮助。有什么我可以为你解答的问题吗？

📊 Token 使用：215 (输入: 11, 输出: 204)


In [8]:
# 基础对话 - 常识问答
response = client.chat.completions.create(
    model="glm-4.7-flash",
    messages=[
        {"role": "user", "content": "请简要介绍人工智能的发展历程"}
    ],
    max_tokens=2048,
    temperature=0.7
)

print("🤖 模型回复：")
print(response.choices[0].message.content)

RateLimitError: Error code: 429 - {'error': {'code': '1302', 'message': '您的账户已达到速率限制，请您控制请求频率'}}

In [ ]:
# 基础对话 - 代码生成（Agentic Coding 场景）
response = client.chat.completions.create(
    model="glm-4.7-flash",
    messages=[
        {
            "role": "system", 
            "content": "你是一个专业的Python编程助手，擅长编写清晰、高效的代码"
        },
        {
            "role": "user", 
            "content": "请写一个快速排序算法，并添加详细的中文注释，解释每一行代码的作用"
        }
    ],
    max_tokens=4096,
    temperature=0.3
)

print("🤖 代码生成结果：")
print(response.choices[0].message.content)

## 3. 多轮对话测试

In [ ]:
# 多轮对话会话管理类
class ChatSession:
    """
    多轮对话会话管理
    维护对话上下文，支持系统提示词
    """
    
    def __init__(self, client, model, system_prompt=None, **default_params):
        self.client = client
        self.model = model
        self.messages = []
        self.default_params = default_params
        
        if system_prompt:
            self.messages.append({"role": "system", "content": system_prompt})
    
    def send(self, user_message, **kwargs):
        """
        发送消息并获取回复
        
        Args:
            user_message: 用户消息
            **kwargs: 额外的请求参数（覆盖默认参数）
        
        Returns:
            tuple: (assistant_message, response)
        """
        self.messages.append({"role": "user", "content": user_message})
        
        # 合并参数
        params = {**self.default_params, **kwargs}
        
        response = self.client.chat.completions.create(
            model=self.model,
            messages=self.messages,
            **params
        )
        
        assistant_message = response.choices[0].message
        self.messages.append({
            "role": "assistant", 
            "content": assistant_message.content
        })
        
        return assistant_message, response
    
    def get_history(self):
        """获取对话历史"""
        return self.messages.copy()
    
    def clear(self, keep_system=True):
        """清空对话历史"""
        if keep_system:
            self.messages = [m for m in self.messages if m["role"] == "system"]
        else:
            self.messages = []
        print("✅ 对话历史已清空")

print("✅ ChatSession 类已定义")

In [ ]:
# 创建多轮对话会话
session = ChatSession(
    client=client,
    model="glm-4.7-flash",
    system_prompt="你是一个Python编程专家，擅长用通俗易懂的语言解释编程概念",
    max_tokens=2048,
    temperature=0.7
)

print("✅ 多轮对话会话已创建")

In [ ]:
# 第1轮：询问递归概念
print("📝 第1轮：什么是递归？\n")
msg, resp = session.send("什么是递归？请用简单的语言解释")
print(f"🤖 助手：{msg.content}\n")
print(f"📊 Token使用：{resp.usage.total_tokens}\n")
print("="*60)

In [ ]:
# 第2轮：要求示例（测试上下文保持）
print("📝 第2轮：能给我一个Python递归的例子吗？\n")
msg, resp = session.send("能给我一个Python递归的例子吗？")
print(f"🤖 助手：{msg.content}\n")
print("="*60)

In [ ]:
# 第3轮：追问优化（测试上下文理解）
print("📝 第3轮：这个例子的时间复杂度是多少？如何优化？\n")
msg, resp = session.send("这个例子的时间复杂度是多少？如何优化？")
print(f"🤖 助手：{msg.content}\n")
print(f"💬 当前对话轮数：{len([m for m in session.messages if m['role'] == 'user'])}")

## 4. 思考模式测试

GLM-4.7-Flash 支持通过 `thinking` 参数启用思考模式，模型会返回 `reasoning_content`（思维链）。

In [ ]:
# 测试：启用思考模式
print("🧠 测试思考模式 - 数学推理问题\n")

response = client.chat.completions.create(
    model="glm-4.7-flash",
    messages=[
        {
            "role": "user", 
            "content": "如果 3 个工人 3 小时可以建 3 面墙，那么 6 个工人 6 小时可以建几面墙？请详细解释推理过程。"
        }
    ],
    thinking={
        "type": "enabled"  # 启用思考模式
    },
    max_tokens=4096
)

message = response.choices[0].message

# 检查是否有思维链
if hasattr(message, 'reasoning_content') and message.reasoning_content:
    print("🧠 思维链（推理过程）：")
    print("-" * 60)
    print(message.reasoning_content)
    print("-" * 60)
    print("\n")

print("💡 最终答案：")
print(message.content)

In [ ]:
# 对比：关闭思考模式
print("🔕 关闭思考模式对比\n")

response_normal = client.chat.completions.create(
    model="glm-4.7-flash",
    messages=[
        {
            "role": "user", 
            "content": "如果 3 个工人 3 小时可以建 3 面墙，那么 6 个工人 6 小时可以建几面墙？"
        }
    ],
    thinking={
        "type": "disabled"  # 关闭思考模式
    },
    max_tokens=2048
)

print("💡 关闭思考模式后的回答：")
print(response_normal.choices[0].message.content)

## 5. 流式输出测试

In [ ]:
# 流式输出测试
print("🌊 流式输出测试\n")
print("🤖 助手：", end="", flush=True)

response = client.chat.completions.create(
    model="glm-4.7-flash",
    messages=[
        {
            "role": "user", 
            "content": "请写一首关于春天的短诗"
        }
    ],
    stream=True,  # 启用流式输出
    max_tokens=1024
)

full_content = ""

for chunk in response:
    if chunk.choices[0].delta.content:
        content = chunk.choices[0].delta.content
        print(content, end="", flush=True)
        full_content += content

print("\n\n✅ 流式输出完成")

In [ ]:
# 流式输出 + 思考模式
print("🌊🧠 流式输出 + 思考模式\n")

response = client.chat.completions.create(
    model="glm-4.7-flash",
    messages=[
        {
            "role": "user", 
            "content": "请解释：为什么天空是蓝色的？"
        }
    ],
    stream=True,
    extra_body={"thinking": {"type": "enabled"}},
    max_tokens=2048
)

print("🧠 思维链：")
print("-" * 60)
reasoning_parts = []
content_parts = []

for chunk in response:
    delta = chunk.choices[0].delta
    
    # 获取思维链
    if hasattr(delta, 'reasoning_content') and delta.reasoning_content:
        print(delta.reasoning_content, end="", flush=True)
        reasoning_parts.append(delta.reasoning_content)
    
    # 获取最终内容
    if delta.content:
        content_parts.append(delta.content)

print("\n" + "-" * 60)
print("\n💡 最终答案：")
print("".join(content_parts))

## 6. 错误处理测试

In [ ]:
# 测试1：无效 API Key
try:
    bad_client = OpenAI(api_key="invalid-key", base_url=BASE_URL)
    bad_client.chat.completions.create(
        model="glm-4.7-flash",
        messages=[{"role": "user", "content": "test"}]
    )
except Exception as e:
    print(f"✅ 无效 API Key 错误捕获：{type(e).__name__}")
    print(f"   错误：{str(e)[:80]}...")

In [ ]:
# 测试2：无效模型名称
try:
    client.chat.completions.create(
        model="invalid-model-name",
        messages=[{"role": "user", "content": "test"}]
    )
except Exception as e:
    print(f"✅ 无效模型错误捕获：{type(e).__name__}")
    print(f"   错误：{str(e)[:80]}...")

In [ ]:
# 测试3：空消息列表
try:
    client.chat.completions.create(
        model="glm-4.7-flash",
        messages=[]
    )
except Exception as e:
    print(f"✅ 空消息错误捕获：{type(e).__name__}")
    print(f"   错误：{str(e)[:80]}...")

In [10]:
# 测试4：超长输入
try:
    long_text = "这是一个测试。" * 50000  # 约10万字
    client.chat.completions.create(
        model="glm-4.7-flash",
        messages=[{"role": "user", "content": long_text}]
    )
except Exception as e:
    print(f"✅ 超长输入错误捕获：{type(e).__name__}")
    print(f"   错误：{str(e)[:80]}...")

✅ 超长输入错误捕获：RateLimitError
   错误：Error code: 429 - {'error': {'code': '1302', 'message': '您的账户已达到速率限制，请您控制请求频率'}}...


## 7. 总结

In [ ]:
print("""
╔════════════════════════════════════════════════════════════════╗
║                    GLM-4.7-Flash 测试总结                      ║
╠════════════════════════════════════════════════════════════════╣
║ 模型规格：                                                      ║
║   • 输入模态：文本                                              ║
║   • 输出模态：文本                                              ║
║   • 上下文窗口：200K tokens                                     ║
║   • 最大输出：128K tokens                                       ║
╠════════════════════════════════════════════════════════════════╣
║ 能力支持：                                                      ║
║   ✅ 思考模式 - 支持 reasoning_content（思维链）                 ║
║   ✅ 流式输出 - 实时响应                                        ║
║   ✅ Function Call - 工具调用                                   ║
║   ✅ 上下文缓存 - 长对话优化                                     ║
║   ✅ 结构化输出 - JSON 格式                                     ║
║   ✅ MCP - 外部工具调用                                         ║
╠════════════════════════════════════════════════════════════════╣
║ 使用建议：                                                      ║
║   • 需要深度推理 → 启用 thinking={type: enabled}                ║
║   • 实时交互场景 → 使用 stream=true                             ║
║   • 长文档处理 → 利用 200K 上下文                               ║
║   • 代码生成 → 适合 Agentic Coding 场景                         ║
╚════════════════════════════════════════════════════════════════╝
""")